In [30]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [39]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.kaiming_uniform_(m.weight)

In [40]:
MP = os.path.join(os.pardir, 'metadata1')

In [41]:
def get_columname(PATH):
    for i in list(os.listdir(PATH)):
        file = os.path.join(PATH, i)
        instance = np.load(file, allow_pickle=True)
        column = []
        column += ['z_mu_' + str(j) for j in range(len(instance['z_mu']))]
        column += ['z_sig_' + str(j) for j in range(len(instance['z_sig']))]
        column += ['vp' + str(j) for j in range(len(instance['vp']))]
        column.append('y')
        return column

def load_meta(PATH):
    total = []
    for i in list(os.listdir(PATH)):
        instance_list= []
        file = os.path.join(PATH, i)
        instance = np.load(file, allow_pickle=True)
        instance_list.extend(instance['z_mu'])
        instance_list.extend(instance['z_sig'])
        instance_list.extend(instance['vp'])
        instance_list.append(int(instance['y'].item()))
        total.append(instance_list)
    
    total = np.array(total)
    df = pd.DataFrame(data = total, columns=get_columname(PATH))
    df = df.astype({'vp0':int, 'vp1':int, 'vp2':int, 'y':int})
    return df
        

In [223]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm



class CustomTrain(nn.Module):
    def __init__(self, n_features, n_classes, layers_list, activation=nn.ReLU(), dropout_list=None, batch_norm=True):
        super(CustomTrain, self).__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.layers_list = layers_list
        self.activation = activation
        self.dropout_list = dropout_list
        self.batch_norm = batch_norm
        self.net = []
        self.b_list = []
        if self.dropout_list:
            self.dropout_list = [nn.Dropout(i) for i in self.dropout_list]
        for i in range(len(self.layers_list)):
            if i==0:
                self.b_list.append(nn.BatchNorm1d(self.n_features))
                self.net.append(nn.Linear(self.n_features, self.layers_list[i]))
            else:
                self.b_list.append(nn.BatchNorm1d(self.layers_list[i-1]))
                self.net.append(nn.Linear(self.layers_list[i-1], self.layers_list[i]))
        self.last_layer = nn.Linear(self.layers_list[-1], self.n_classes)
        self.net = nn.ModuleList(self.net)
        
    
    def forward(self, x):

        for i, l in enumerate(self.net):
            #print(i)
            if self.batch_norm:
                x = (self.b_list[i])(x)
            x.to(device)
            x = self.activation(l(x))
            
            if self.dropout_list:
                x = (self.dropout_list[i])(x)
            
        x = self.last_layer(x)
        if self.n_classes == 1:
            x = torch.sigmoid(x)
   
        return x

class datapaltas(Dataset):
    def __init__(self, df, scale =True, y_idx=-1):
        self.df = df
        self.y_idx = y_idx

    def __getitem__(self, index):
        X = (self.df.iloc[index,:self.y_idx]).values
        X = X.astype(np.float64)
        X = torch.from_numpy(X).float()
        y = self.df.iloc[index,self.y_idx]
        #print(y)
        y = torch.Tensor([y]).long()
        return {'x':X, 'y':y}
    
    def __len__(self):
        return self.df.shape[0]
    
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

In [224]:
device = 'cpu'
def train(data, model, ep = 120, save=False, prefix=None):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)
    idx_result = {}
    a = datapaltas(df = data, y_idx=-1)
    EPOCHS = ep
    skf = StratifiedKFold(shuffle=True)
    ## y_idx -2
    skf.get_n_splits(data, data.iloc[:,-1])
    vec_train = np.array([train_ids for train_ids,_ in skf.split(data, data.iloc[:,-1])])
    vec_test = np.array([test_ids for _,test_ids in skf.split(data, data.iloc[:,-1])])
    acv, tav = [], []
    foldn = 0
    for train_ids, test_ids in zip(vec_train, vec_test):
        acv_, tav_ = [], []
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        train_loader = torch.utils.data.DataLoader(
                            a, 
                            batch_size=64, sampler=train_subsampler)
        test_loader = torch.utils.data.DataLoader(
                            a,
                            batch_size=1, sampler=test_subsampler)

        EPOCHS = ep
        itt = tqdm(range(EPOCHS))
        for i in itt:
            loss_epoch = 0
            acc_train, acc_test = 0.0, 0.0
            model.train()
            for sample in train_loader:
                X, y = sample['x'], sample['y']
                X, y = X.to(device), y.to(device).flatten()
                y_pred = model(X)
                #print(y_pred, y)
                loss = criterion(y_pred, y)
                acc = multi_acc(y_pred, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_epoch += loss.item()
                acc_train += acc.item()
                
            
            model.eval()
            
            with torch.no_grad():
                for sample in test_loader:
                    X_test, y_test = sample['x'], sample['y']
                    X_test, y_test = X_test.to(device), y_test.to(device).flatten()
                    print(model.training)
                    y_hat = model(X_test)
                    acc = multi_acc(y_hat, y_test)
                    acc_test += acc.item()
                
            acv_.append(acc_test/len(test_loader))
            tav_.append(acc_train/len(train_loader))
            #print(acv_)
            if save and (abs(acc_test/len(test_loader) - max(acv_)<1e-8)):
                    if os.path.exists(os.path.join(os.curdir, prefix+"model_fold_"+str(foldn)+".pth")):
                        os.remove(os.path.join(os.curdir, prefix+"model_fold_"+str(foldn)+".pth"))
                    torch.save(model, os.path.join(os.curdir, prefix+"model_fold_"+str(foldn)+".pth"))
                    itt.set_postfix({'epoch_model': i, 'best_acc':max(acv_)})
            t1, t2 = acc_train/len(train_loader), acc_test/len(test_loader)
            itt.set_description(f"Acc train: {t1:.2f} Acc test: {t2:.2f}")
        acv.append(acv_)
        tav.append(tav_)
        

        foldn += 1
        model.apply(init_normal)
        
    acv = np.array(acv)
    tav = np.array(tav)
    idx_result['acc'] = np.mean(acv, axis = 0)
    idx_result['acc_std'] = np.std(acv, axis=0)
    idx_result['train_acc'] = np.mean(tav, axis=0)
    return idx_result

In [225]:
a = load_meta(MP)

In [226]:

modelo = CustomTrain(67, 788, [128, 128, 256, 256, 256], nn.GELU(), batch_norm=True)
modelo.to(device)

CustomTrain(
  (activation): GELU()
  (last_layer): Linear(in_features=256, out_features=788, bias=True)
  (net): ModuleList(
    (0): Linear(in_features=67, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): Linear(in_features=256, out_features=256, bias=True)
  )
)

In [227]:
eo = train(a, modelo, ep=50, save=True, prefix='met1_v4')

C:\Users\LENOVO\.conda\envs\paltas\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\LENOVO\AppData\Local\Temp/ipykernel_15048/4271564252.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vec_train = np.array([train_ids for train_ids,_ in skf.split(data, data.iloc[:,-1])])
C:\Users\LENOVO\.conda\envs\paltas\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\LENOVO\AppData\Local\Temp/ipykernel_15048/4271564252.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tupl

False


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 67])

In [ ]:
plt.plot(eo["acc"])
plt.plot(eo["train_acc"])

In [202]:
for child in model.children():
    print(child)

GELU()
Linear(in_features=256, out_features=788, bias=True)
ModuleList(
  (0): Linear(in_features=67, out_features=1024, bias=True)
  (1): Linear(in_features=1024, out_features=1024, bias=True)
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): Linear(in_features=512, out_features=256, bias=True)
)


In [193]:
m = nn.BatchNorm1d(100)
m.eval()
m(torch.randn(1,100))

tensor([[ 0.8590,  0.5250,  0.7132, -0.0781,  0.5314,  0.3811,  0.6054,  1.6083,
          0.4533,  0.4299, -1.1930, -0.0554, -0.2127, -1.0761,  0.5811,  0.5002,
          0.2879,  1.3363,  1.2084, -1.0590, -0.2013, -0.1151, -0.3977, -0.7007,
          0.6692,  0.9484, -1.7068,  0.5066, -0.9476,  0.1339,  0.7815,  0.0863,
         -0.4813,  0.1134, -0.0872, -1.5166, -1.5565,  2.3727,  1.3591,  0.6989,
          0.7292,  0.9913, -0.1311, -0.2473, -1.3831,  0.7652, -0.2156,  0.8213,
          0.0957, -0.5370, -0.2610,  0.3676, -0.2735,  2.1165,  0.9705, -1.2300,
          0.6576,  1.2233, -0.5727,  0.8554,  0.6659, -0.4522, -0.3276, -0.3109,
         -0.3851, -0.6309, -0.2510,  0.9572,  1.0285, -0.8394, -0.2947,  0.2623,
         -1.6794, -0.2820, -1.6820, -0.0750,  0.3905, -0.7908,  0.0696, -0.7503,
         -2.0633,  1.5216,  1.1269, -0.5935,  1.5394, -1.6033, -0.3636,  0.4839,
         -0.8443,  0.9877,  0.2262,  0.6526, -0.5637,  0.3416, -1.4988,  0.9133,
         -1.0327,  2.1566, -